# GPT 

На прошлой паре мы разбирали транформер в целом и попробовали дообучать BERT для задачи классификации. На этом посмотрим на другую популярную транформерную модель -  GPT (Generative pretrained transformer).

## Чем BERT и GPT похожи?
- Обе модели основаны на транформерных блоках с механизмом внимания и позиционным кодированием. Есть небольшие отличия в некоторых частях моделей (в gpt, например используется sparse attention), но они не принципиальные.

- В обоих моделях используется BPE токенизация. 
- Обе модели решают задачу языкового моделирования.
- В обеих моделях используется только self-attention. 

## Чем отличаются BERT и GPT?
- BERT - это энкодер модель. А GPT - это декодер модель.
- BERT обучается на задачах Мasked language modelling (MLM) и Next Sentence Prediction (NSP). Это искусственные self supervised задачи: в MLM модель должна заполнить пропуски в тексте (например, "Я [MASK] в [MASK] один." -> "Я пошел в кино один"), а NSP - определить являются ли два предложения соседними. А GPT - это авторегрессионная модель, которая учится продолжать данную ей последовательностью ("Я пошел.." -> "Я пошел в" -> "Я пошел в кино" -> "Я пошел в кино один")
- BERT смотрит на входной текст целиком, а в GPT attention для каждого отдельного токена расчитывается только по предудущим токенам.  
- В GPT (особенно в последней версии огромных размеров) целевой задачей является именно генерация текста и поэтому предобученную модель либо вообще никак не дообучают, либо дообучают на генерацию нужных текстов (а не конкретную задачу вроде классификации). BERT используется в основном только для эмбеддингов и дообучения на конкретные задачи (классификация), заполнение пропусков в практических задачах практически бесполезно. 

### Главная особенность GPT
Предобученную GPT можно использовать для решения целевой задачи (той же классификации) сразу из коробки, без дообучения, через генерацию текста. Нужно только сформулировать входную последовательность подходящим образом. Например, для определения класса текста можно подать в модель нужный текст и добавить в конце "Тема этого текста: ", на что модель выдаст продолжение "Новости" или "Спорт". Такой подход называется zero-shot learning. 
Еще есть few-shot learning, когда в модель подается что-то вроде "2+2=4, 1+3=4, 1+2= ", то есть модели показывается несколько примеров того, что от нее хотят и просят анологично продолжить последний пример. В этом случае также не происходит дообучения модели (веса не обновляются), поэтому на каждом запуске в модель нужно подавать нужный контекст. Такие контексты называются затравками (prompts). Подбирать их может быть достаточно сложно, даже есть такой термин "prompt engineering". (Есть способ обойти ручную подброку затравок, который называется prompt tuning. Про него можно почитать вот тут - https://habr.com/ru/company/yandex/blog/588214/) 

Также важная характеристика GPT моделей - их огромный размер. Самая большая модель GPT-3 имеет 175 млрд параметров, и она даже не выложена в открытый доступ, а предоставляется через API (https://openai.com/api/). Есть варианты поменьше, в том числе и для русского (RuGPT-3 от Сбера). Их можно попробовать в коллабе через huggingface. Давайте попробуем!



In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 40.8 MB/s 
     |████████████████████████████████| 6.6 MB 33.4 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 37.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# Загружаем модель ruGPT от сбера
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

In [4]:
# prompt engineering 
text = "Вопрос: 'Сколько будет 2+2?'\n Ответ: " # работает
text = "Вопрос: 'Сколько будет 3+3?'\n Ответ: " # не очень работает
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=False) 

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Вопрос: 'Сколько будет 3+3?'
 Ответ: 
3+3=5


## Методы генерации текста
В вводном семинаре про языковое моделирование мы уже пробовали генерировать текст и там мы сталкивались с вопросом - а как выбирать конкретный токен по вероятностям, которые выдает модель? Если просто использовать .argmax(), то генерация будет слишком детерменированной и будет много повторов. Обычно это не то, что нам нужно. GPT может применяться, например, для генерации ответов в чатботе и однобразность ответов - сильный минус.

На семинаре про базовое языковое моделирование мы решали это случайным семплированием из полученного распределения, но на практике используют немного другие подходы. Давайте на них посмотрим.

In [5]:
# возьмем какой-нибудь текст
text = 'За окном дождь. Холодный и противный. Хочется  '
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

### Argmax
Мы уже выше попробовали аргмакс, но давайте проверим его и на новом тексте, чтобы потом было с чем сравнивать

In [6]:
# если повторить запуск результат не изменится
out = model.generate(input_ids, do_sample=False, max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   спрятаться  под  одеяло,  укрыться  от  дождя,  укрыться  от  него,  укрыться  от  него,  укрыться  от  него,  укрыться  от


### Beam Search

Чтобы разнообразить генерацию можно на каждом шаге выбирать не 1 наиболее вероятный, а N наиболее вероятных и генерировать продолжения для них всех.Такой подход называется beam search (поиск пучком). Параметр N (размер пучка, beam size) настраивается, но поставить его слишком большим не получится, т.к. будет слишком много комбинаций на длинных текстах. Выбрать итоговый текст можно по общей вероятности, а можно просто рандомно.

In [7]:
# beam search уже реализован в hg поэтому нужно только задать параметр num_beams
out = model.generate(input_ids, do_sample=False, num_beams=10, max_length=60)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   спрятаться  под  одеяло  и  закрыть  глаза,  чтобы  не  видеть,  не  слышать,  не  чувствовать,  чтобы  не  видеть,  не  слышать,  не  чувствовать,  чтобы


Но это тоже не очень сильно помогает

### Сэмплирование с Температурой
Самый распространенный подход - семплирование с температурой. Это очень похоже на то, что мы делали на семинаре по языковому моделированию (np.random.choice(words, p=probas), но здесь добавляется параметр, который регулирует насколько случайно или детерменировано мы будем выбирать следующий токен. Нулевая температура означает, что мы на каждом шаге просто выбираем по argmax(), а очень большая температура будет приводить к полному рандому. Под конкретную задачу температуру нужно подбирать отдельно, можно начать с 0 и постепенно увеличивать, смотря на получаемое разнобразие.

(температурой это называется потому что формула взята из физических уравнений, где этот параметр действительно отвечает за температуру)

In [8]:
out = model.generate(input_ids, do_sample=True, temperature=0.01, max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   спрятаться  под  одеяло,  укрыться  от  дождя,  укрыться  от  него,  укрыться  от  него,  укрыться  от  него,  укрыться  от


In [9]:
out = model.generate(input_ids, do_sample=True, temperature=0.3, max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   спрятаться  под   одеяло,   укрыться   от
дождя, от холода.

- Ты не спишь, - говорит мама. - Что-то случилось?




In [10]:
out = model.generate(input_ids, do_sample=True, temperature=0.8, max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   погреться  у  жаркого  огня,  но   не  могу  позволить
вытащить из-под кровати какую-нибудь железяку,  а  в 


In [11]:
out = model.generate(input_ids, do_sample=True, temperature=1.5, max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   обсушить-прогнать-намочить волосы) Но я не имею даже опыта сушить/погружаться. И нет, пока нет) Пошля-я так


Видно, что чем больше температура, тем случайнее текст


Чтобы не выбирать совсем маловероятные слова, можно указать дополнительный параметр - top_k. Так семплирование будет происходить только из K наиболее вероятных токенов

In [12]:
out = model.generate(input_ids, 
                     do_sample=True,
                     temperature=1.2,
                     top_k=20,
                     max_length=50,
                    )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



За окном дождь. Холодный и противный. Хочется   погреться, а тут  еще   мерзнут   руки-то.  Надо  взять  телефон  и
приехать в редакцию. Там у редактора  должна быть газета. Я


Температура большая, но текст все равно получается более менее тематически связанный

# Дообучение GPT

Чтобы генерировать тексты в нужном стиле/домене, можно дообучить GPT на своем корпусе. Давайте попробуем это сделать

In [13]:
# возьмем модель поменьше, так как дообучение это обновление весов
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]


Для наглядности результата возьмем начало Евгения Онегина

In [14]:
text = """I
«Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог.
Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отходя ни шагу прочь!
Какое низкое коварство
Полуживого забавлять,
Ему подушки поправлять,
Печально подносить лекарство,
Вздыхать и думать про себя:
Когда же черт возьмет тебя!»
II
		Так думал молодой повеса,
Летя в пыли на почтовых,
Всевышней волею Зевеса
Наследник всех своих родных. —
Друзья Людмилы и Руслана!
С героем моего романа
Без предисловий, сей же час
Позвольте познакомить вас:
Онегин, добрый мой приятель,
Родился на брегах Невы,
Где, может быть, родились вы
Или блистали, мой читатель;
Там некогда гулял и я:
Но вреден север для меня[3 - Писано в Бесарабии.].
III
		Служив отлично-благородно,
Долгами жил его отец,
Давал три бала ежегодно
И промотался наконец.
Судьба Евгения хранила:
Сперва Madame за ним ходила,
Потом Monsieur ее сменил;
Ребенок был резов, но мил.
Monsieur l’Abbe€, француз убогой,
Чтоб не измучилось дитя,
Учил его всему шутя,
Не докучал моралью строгой,
Слегка за шалости бранил
И в Летний сад гулять водил.
IV
		Когда же юности мятежной
Пришла Евгению пора,
Пора надежд и грусти нежной,
Monsieur прогнали со двора.
Вот мой Онегин на свободе;
Острижен по последней моде;
Как dandy[4 - Dandy, франт.] лондонский одет —
И наконец увидел свет.
Он по-французски совершенно
Мог изъясняться и писал;
Легко мазурку танцевал
И кланялся непринужденно;
Чего ж вам больше? Свет решил,
Что он умен и очень мил.
V
		Мы все учились понемногу
Чему-нибудь и как-нибудь,
Так воспитаньем, слава богу,
У нас немудрено блеснуть.
Онегин был, по мненью многих
(Судей решительных и строгих),
Ученый малый, но педант[5 - Педант– здесь: «человек, выставляющий напоказ свои знания, свою ученость, с апломбом, судящий обо всем». (Словарь языка А. С. Пушкина.)].
Имел он счастливый талант
Без принужденья в разговоре
Коснуться до всего слегка,
С ученым видом знатока
Хранить молчанье в важном споре
И возбуждать улыбку дам
Огнем нежданных эпиграмм.
VI
		Латынь из моды вышла ныне:
Так, если правду вам сказать,
Он знал довольно по-латыни,
Чтоб эпиграфы разбирать,
Потолковать об Ювенале,
В конце письма поставить vale[6 - Vale – будь здоров (лат.).],
Да помнил, хоть не без греха,
Из Энеиды два стиха.
Он рыться не имел охоты
В хронологической пыли
Бытописания земли;
Но дней минувших анекдоты,
От Ромула до наших дней,
Хранил он в памяти своей.
VII
		Высокой страсти не имея
Для звуков жизни не щадить,
Не мог он ямба от хорея,
Как мы ни бились, отличить.
Бранил Гомера, Феокрита;
Зато читал Адама Смита
И был глубокий эконом,
То есть умел судить о том,
Как государство богатеет,
И чем живет, и почему
Не нужно золота ему,
Когда простой продукт имеет.
Отец понять его не мог
И земли отдавал в залог.
VIII
		Всего, что знал еще Евгений,
Пересказать мне недосуг;
Но в чем он истинный был гений,
Что знал он тверже всех наук,
Что было для него измлада
И труд, и мука, и отрада,
Что занимало целый день
Его тоскующую лень, —
Была наука страсти нежной,
Которую воспел Назон,
За что страдальцем кончил он
Свой век блестящий и мятежный
В Молдавии, в глуши степей,
Вдали Италии своей.
X
		Как рано мог он лицемерить,
Таить надежду, ревновать,
Разуверять, заставить верить,
Казаться мрачным, изнывать,
Являться гордым и послушным,
Внимательным иль равнодушным!
Как томно был он молчалив,
Как пламенно красноречив,
В сердечных письмах как небрежен!
Одним дыша, одно любя,
Как он умел забыть себя!
Как взор его был быстр и нежен,
Стыдлив и дерзок, а порой
Блистал послушною слезой!
XI
		Как он умел казаться новым,
Шутя невинность изумлять,
Пугать отчаяньем готовым,
Приятной лестью забавлять,
Ловить минуту умиленья,
Невинных лет предубежденья
Умом и страстью побеждать,
Невольной ласки ожидать,
Молить и требовать признанья,
Подслушать сердца первый звук,
Преследовать любовь и вдруг
Добиться тайного свиданья…
И после ей наедине
Давать уроки в тишине!
XII
		Как рано мог уж он тревожить
Сердца кокеток записных!
Когда ж хотелось уничтожить
Ему соперников своих,
Как он язвительно злословил!
Какие сети им готовил!
Но вы, блаженные мужья,
С ним оставались вы друзья:
Его ласкал супруг лукавый,
Фобласа давний ученик,
И недоверчивый старик,
И рогоносец величавый,
Всегда довольный сам собой,
Своим обедом и женой.


"""

В huggingface уже реализованы все инструменты для дообучения. Даже если вы не знакомы с pytorch разобраться будет не сложно

In [15]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## Обучение
Через класс Trainer реализовано все обучение, останется только запустить фит (точнее трейн в случае hg)

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [17]:
trainer.train()

***** Running training *****
  Num examples = 21
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.1191801357269287, metrics={'train_runtime': 77.5904, 'train_samples_per_second': 27.065, 'train_steps_per_second': 1.289, 'total_flos': 68589158400000.0, 'train_loss': 0.1191801357269287, 'epoch': 100.0})

In [20]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет 
С утра до позднего вечера;
Утешает детей,
Пугает зверей и
Печально скулит.
XI
		Как рано он ложился,
Как рано лег!
Как рано ворочался
Под дождем одетый!
Дождь лил целый час:
Не меньше пары капель,
И заря мочила,
И гнали на базар,
На убой.VII
		Вот наконец пришел он к нам; устал очень
И на другой же день
На другой же день пошел,
Чтобы обед приготовить.
В один из таких уж дней
Проехался тут мой д’Артаньян‐девичий род,
Переселившийся на брег,
Иной год быль им.А мой д’Артаньян
Сидел у отца своего;
Молокосос он грубый
Был, однако,
И совсем не плакал;
Ловил минуту уми


Результат не очень хороший, но можно попробовать добавить еще текстов и попробовать другие параметры семплирования